# Load data from data generator and visualize

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from ipywidgets import interactive, widgets, interact

import os
import numpy as np
import matplotlib.pyplot as plt
import modules.data as data

%matplotlib inline

notebook_path = os.path.abspath("notebook_predicted_data_jakob.ipynb")

config = {}
config['data'] = {}
config['data']['folder'] = {}

# config['data']['folder']['train'] = 'data/datasets_train_waveequation'
# config['data']['folder']['test'] = 'data/datasets_test_waveequation'
# config['data']['input_file_prefixes'] = ['field_dataset_', 'domain_dataset_', 'material_dataset_']
# config['data']['augmentation'] = 'mirror_and_flip' 

config['data']['folder']['train'] = 'data/datasets_train_heatequation'
config['data']['folder']['test'] = 'data/datasets_test_heatequation'
config['data']['input_file_prefixes'] = ['field_dataset_', 'domain_surf_dataset_', 'material_dataset_']
config['data']['augmentation'] = 'rotate' 

config['data']['input_sizes'] = [3,1,1]
config['data']['output_size'] = 1
config['data']['batch_size'] = 32
config['data']['file_batch_size'] = 1
config['data']['reference_input'] = 0
config['data']['output_crop'] = 0
config['data']['randomize_files'] = True

abs_path = os.path.dirname(notebook_path)
data_path_train = abs_path+"/"+config['data']['folder']['train']+"/"
data_path_test = abs_path+"/"+config['data']['folder']['test']+"/"
train_files, val_files = data.get_files(data_path_train, data_path_test, config['data']['input_file_prefixes'])

train_data, val_data = data.get_dataset_from_generators(train_files,
                                                        val_files,
                                                        config['data']['input_sizes'],
                                                        config['data']['output_size'],
                                                        config['data']['batch_size'],
                                                        config['data']['file_batch_size'],
                                                        config['data']['reference_input'],
                                                        config['data']['output_crop'],
                                                        config['data']['augmentation'],
                                                        config['data']['randomize_files'])

data, = train_data.take(1)

fig1, axs1 = plt.subplots(1, 10, figsize=(30,3))
fig1.suptitle('Dataset from generator')

im1 = []

im1.append(axs1[0].imshow(data[0]['input_0'].numpy()[0,:,:,0], origin='lower'))
im1.append(axs1[1].imshow(data[0]['input_0'].numpy()[0,:,:,1], origin='lower'))
im1.append(axs1[2].imshow(data[0]['input_0'].numpy()[0,:,:,2], origin='lower'))
im1.append(axs1[3].imshow(data[0]['input_1'].numpy()[0,:,:,0], origin='lower'))
im1.append(axs1[4].imshow(data[0]['input_2'].numpy()[0,:,:,0], origin='lower'))
im1.append(axs1[5].imshow(data[0]['label'].numpy()[0,:,:,0], origin='lower'))
im1.append(axs1[6].imshow(data[1].numpy()[0,:,:,0], origin='lower'))

axs1[0].set_title('Input 0 [0]')
axs1[1].set_title('Input 0 [1]')
axs1[2].set_title('Input 0 [2]')
axs1[3].set_title('Input 1')
axs1[4].set_title('Input 2')
axs1[5].set_title('Input Label')
axs1[6].set_title('Label')
axs1[7].set_title('Diff(Input 0 [2], Input Label)')
axs1[8].set_title('Diff(Input 0 [2], Label)')
axs1[9].set_title('Diff(Input Label, Label)')

if config['data']['output_crop'] <= 0:
    diff_1 = np.abs(data[0]['input_0'].numpy()[0,:,:,2]-data[1].numpy()[0,:,:,0])
    diff_2 = np.abs(data[0]['input_0'].numpy()[0,:,:,2]-data[0]['label'].numpy()[0,:,:,0])
    diff_3 = np.abs(data[1].numpy()[0,:,:,0]-data[0]['label'].numpy()[0,:,:,0])

    im1.append(axs1[7].imshow(diff_1, origin='lower'))
    im1.append(axs1[8].imshow(diff_2, origin='lower'))
    im1.append(axs1[9].imshow(diff_3, origin='lower'))

else:
    diff_1 = np.abs(data[0]['input_0'].numpy()[0,config['data']['output_crop']:-config['data']['output_crop'],config['data']['output_crop']:-config['data']['output_crop'],2]-data[1].numpy()[0,:,:,0])
    diff_2 = np.abs(data[0]['input_0'].numpy()[0,config['data']['output_crop']:-config['data']['output_crop'],config['data']['output_crop']:-config['data']['output_crop'],2]-data[0]['label'].numpy()[0,:,:,0])
    diff_3 = np.abs(data[1].numpy()[0][:,:,0]-data[0]['label'].numpy()[0][:,:,0])

    im1.append(axs1[7].imshow(diff_1, origin='lower'))
    im1.append(axs1[8].imshow(diff_2, origin='lower'))
    im1.append(axs1[9].imshow(diff_3, origin='lower'))

for i in range(len(im1)):
    fig1.colorbar(im1[i], ax=axs1[i])
